## This notebook makes "CMB-augmented" catalogs.

It takes around **10 minutes** to run.

Input files from previous pipeline stages:
  - `xl_ge_gg.txt`: Power spectra $P_{ge}(k)$, $P_{gg}(k)$.
  - `wcmb_90.fits` and `wcmb_150.fits`: CMB pixel weight function $W_{\rm cmb}(\theta)$.
  - `fl_90.txt`, `fl_150.txt`: CMB harmonic-space weight function $F_l$.

First, we read the following catalogs:
  - SDSS galaxy catalog (CMASS_North, DR12)
  - Corresponding random catalog
  - 1000 mock catalogs (CMASS_North, DR12, QPM)
  - Corresponding mock random catalog

Second, for each catalog, we add the following columns:
  - `tcmb_90`: filtered CMB temperature at location of each galaxy, 90 GHz
  - `tcmb_150`: filtered CMB temperature at location of each galaxy, 150 GHz
  - `bv_90`: per-galaxy velocity reconstruction bias $b_v$, 90 GHz
  - `bv_150`: per-galaxy velocity reconstruction bias $b_v$, 150 GHz

where $b_v$ is the per-object velocity reconstruction, which we compute in the approximation (see overleaf):

$$\begin{align}
b_v(\theta,\chi) &\approx B_v(\chi) \, W_{\rm CMB}(\theta) \\
B_v(\chi) &= \frac{K(\chi)}{\chi^2} \sum_l \frac{2l+1}{4\pi} b_l F_l \, P_{ge}^{\rm true}(k,\chi)_{k=l/\chi} \\
K(z) &= -T_{\rm CMB} \, \sigma_T \, n_{e0} \, x_e(z) \, e^{-\tau(z)} \, (1+z)^2
\end{align}$$

Third, we write the catalogs to disk in an hdf5 file format:
  - Galaxy catalog -> `catalogs/galaxies.h5`
  - Random catalog -> `catalogs/randoms.h5`
  - Mock catalog -> `catalogs/mock_{i}.h5`, where $0 \le i < 1000$.
  - Mock random catalog -> `catalogs/mock_randoms.h5`
    
Finally, we write two miscellaneous files that will be convenient in later pipeline stages:
  - `bounding_box.pkl`: bounding box of random catalog (instance of class `kszx.BoundingBox`)
  - `ngal_mock.npy`: 1-d numpy array of length nmocks, containing number of galaxies in each mock

After this script has been run, we don't need to process the CMB maps (or CMB data products such as $W_{\rm cmb}(\theta)$, $F_l$) again. Subsequent pipeline stages can work entirely with "augmented" catalogs, with a few numbers $T_{90}, T_{150}, b_v^{90}, b_v^{150}$ per galaxy.

## Imports and global variables

In [1]:
import kszx
import pixell
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# File 'global_params.py' in current directory
import global_params

lmax = global_params.ksz_lmax
act_dr = global_params.act_dr

survey = global_params.sdss_survey
rpad = global_params.sdss_rpad
pixsize = global_params.sdss_pixsize
zmin = global_params.sdss_zmin
zmax = global_params.sdss_zmax
zeff = global_params.sdss_zeff
mock_type = global_params.sdss_mock_type
nmocks = global_params.sdss_nmocks

In [3]:
cosmo = kszx.Cosmology('planck18+bao')

Running CAMB


## Read input files and process CMB maps

In [4]:
def read_cl(filename, col):
    fl = np.loadtxt(filename)
    assert fl.ndim == 2
    assert len(fl) == lmax+1
    assert np.all(fl[:,0] == np.arange(lmax+1))
    return fl[:,col]

xl_ge = read_cl('xl_ge_gg.txt', col=1)

In [5]:
def prepare_freq(freq):
    """Returns pair of pixell maps (tcmb, bv)."""

    bl = kszx.act.read_beam(freq, dr=5, lmax=lmax)
    fl = read_cl(f'fl_{freq}.txt', col=1)

    l = np.arange(lmax+1)
    B = np.sum((2*l+1)/(4*np.pi) * bl * fl * xl_ge)
    B *= cosmo.K(z=zeff) / cosmo.chi(z=zeff)**2
    
    wcmb = kszx.pixell_utils.read_map(f'wcmb_{freq}.fits')
    bv = B * wcmb
    
    tcmb = kszx.act.read_cmb(freq, dr=act_dr, download=True)
    tcmb *= wcmb
    alm = kszx.pixell_utils.map2alm(tcmb, lmax)
    alm = pixell.curvedsky.almxfl(alm, fl)
    tcmb = kszx.pixell_utils.alm2map(alm, tcmb.shape, tcmb.wcs)

    return tcmb, bv

In [6]:
tcmb_90, bv_90 = prepare_freq(90)
tcmb_150, bv_150 = prepare_freq(150)

kszx: environment variable $KSZX_DATA_DIR not defined, using /home/kmsmith/kszx_data instead
Reading /home/kmsmith/kszx_data/act/dr5.01/beams/act_planck_dr5.01_s08s18_f090_daynight_beam.txt
Reading wcmb_90.fits
Reading /home/kmsmith/kszx_data/act/dr5.01/act_planck_dr5.01_s08s18_AA_f090_daynight_map_srcfree.fits
Reading /home/kmsmith/kszx_data/act/dr5.01/beams/act_planck_dr5.01_s08s18_f150_daynight_beam.txt
Reading wcmb_150.fits
Reading /home/kmsmith/kszx_data/act/dr5.01/act_planck_dr5.01_s08s18_AA_f150_daynight_map_srcfree.fits


## Define function process_catalog() that will be run on all catalogs/randoms/mocks

In [7]:
def process_catalog(catalog, output_filename):
    catalog.apply_redshift_cut(zmin, zmax)

    cols = [ ('tcmb_90',tcmb_90), ('tcmb_150',tcmb_150), ('bv_90',bv_90), ('bv_150',bv_150) ]
    
    for col_name, pixell_map in cols:
        t = kszx.pixell_utils.eval_map_on_catalog(pixell_map, catalog, pad=0.0)
        catalog.add_column(col_name, t)
    
    catalog.write_h5(output_filename)

## Process galaxy catalog

In [8]:
gcat = kszx.sdss.read_galaxies(survey, download=True)
process_catalog(gcat, 'catalogs/galaxies.h5')

Reading /home/kmsmith/kszx_data/sdss/DR12v5/galaxy_DR12v5_CMASS_North.fits
Read 618806 galaxies from /home/kmsmith/kszx_data/sdss/DR12v5/galaxy_DR12v5_CMASS_North.fits, columns ['ra_deg', 'dec_deg', 'z', 'wfkp', 'wcp', 'wzf', 'wsys', 'cboss', 'id']
Reminder: you probably want to call cat.apply_redshift_cut(zmin,zmax) on the Catalog returned by kszx.sdss.read_galaxies())
Redshift cut: zmin=0.43 zmax=0.7: 618806 -> 568776 [91.915% retained]
Writing catalogs/galaxies.h5
Wrote 568776 galaxies to catalogs/galaxies.h5, columns ['ra_deg', 'dec_deg', 'z', 'wfkp', 'wcp', 'wzf', 'wsys', 'cboss', 'id', 'tcmb_90', 'tcmb_150', 'bv_90', 'bv_150']


## Process random catalog (and write bounding box)

In [9]:
rcat = kszx.sdss.read_randoms(survey, download=True)
process_catalog(rcat, 'catalogs/randoms.h5')

box = kszx.BoundingBox(rcat.get_xyz(cosmo), pixsize, rpad)
kszx.io_utils.write_pickle('bounding_box.pkl', box)
print('\n', box)

Reading /home/kmsmith/kszx_data/sdss/DR12v5/random0_DR12v5_CMASS_North.fits
Read 32151741 galaxies from /home/kmsmith/kszx_data/sdss/DR12v5/random0_DR12v5_CMASS_North.fits, columns ['ra_deg', 'dec_deg', 'z', 'wfkp']
Reading /home/kmsmith/kszx_data/sdss/DR12v5/random1_DR12v5_CMASS_North.fits
Read 32151250 galaxies from /home/kmsmith/kszx_data/sdss/DR12v5/random1_DR12v5_CMASS_North.fits, columns ['ra_deg', 'dec_deg', 'z', 'wfkp']
Reminder: you probably want to call cat.apply_redshift_cut(zmin,zmax) on the Catalog returned by kszx.sdss.read_randoms())
Redshift cut: zmin=0.43 zmax=0.7: 64302991 -> 59175065 [92.025% retained]
Writing catalogs/randoms.h5
CMASS_North randoms: Wrote 59175065 galaxies to catalogs/randoms.h5, columns ['ra_deg', 'dec_deg', 'z', 'wfkp', 'tcmb_90', 'tcmb_150', 'bv_90', 'bv_150']
Writing bounding_box.pkl

 BoundingBox(
    npix = [320 576 320],
    pixsize = 10.0,
    boxsize = [3200. 5760. 3200.],
    kfund = [0.0019635  0.00109083 0.0019635 ],
    knyq = 0.3141592

## Process mock random catalog

In [10]:
mrcat = kszx.sdss.read_mock_randoms(survey, mock_type, download=True)
process_catalog(mrcat, 'catalogs/mock_randoms.h5')

Reading /home/kmsmith/kszx_data/sdss/DR12v5/qpm_mocks/mock_random_DR12_CMASS_N_50x1.rdzw
mock_random_DR12_CMASS_N_50x1.rdzw: Read 32292068 galaxies from /home/kmsmith/kszx_data/sdss/DR12v5/qpm_mocks/mock_random_DR12_CMASS_N_50x1.rdzw, columns ['ra_deg', 'dec_deg', 'z', 'wfkp']
Reading /home/kmsmith/kszx_data/sdss/DR12v5/qpm_mocks/mock_random_DR12_CMASS_N_50x2.rdzw
mock_random_DR12_CMASS_N_50x2.rdzw: Read 32298915 galaxies from /home/kmsmith/kszx_data/sdss/DR12v5/qpm_mocks/mock_random_DR12_CMASS_N_50x2.rdzw, columns ['ra_deg', 'dec_deg', 'z', 'wfkp']
Redshift cut: zmin=0.43 zmax=0.7: 64590983 -> 64590983 [100.0% retained]
Writing catalogs/mock_randoms.h5
CMASS_North qpm mock randoms: Wrote 64590983 galaxies to catalogs/mock_randoms.h5, columns ['ra_deg', 'dec_deg', 'z', 'wfkp', 'tcmb_90', 'tcmb_150', 'bv_90', 'bv_150']


## Process mocks (and write mock_ngal.npy)

In [11]:
%%time

mock_ngal = np.zeros(nmocks)

for i in range(nmocks):
    mcat = kszx.sdss.read_mock(survey, mock_type, i, download=True)
    process_catalog(mcat, f'catalogs/mock_{i}.h5')
    mock_ngal[i] = mcat.size

kszx.io_utils.write_npy('mock_ngal.npy', mock_ngal)

Reading /home/kmsmith/kszx_data/sdss/DR12v5/qpm_mocks/mock_galaxy_DR12_CMASS_N_QPM_0001.rdzw
mock_galaxy_DR12_CMASS_N_QPM_0001.rdzw: Read 642051 galaxies from /home/kmsmith/kszx_data/sdss/DR12v5/qpm_mocks/mock_galaxy_DR12_CMASS_N_QPM_0001.rdzw, columns ['ra_deg', 'dec_deg', 'z', 'wfkp', 'wveto']
Redshift cut: zmin=0.43 zmax=0.7: 642051 -> 642051 [100.0% retained]
Writing catalogs/mock_0.h5
mock_galaxy_DR12_CMASS_N_QPM_0001.rdzw: Wrote 642051 galaxies to catalogs/mock_0.h5, columns ['ra_deg', 'dec_deg', 'z', 'wfkp', 'wveto', 'tcmb_90', 'tcmb_150', 'bv_90', 'bv_150']
Reading /home/kmsmith/kszx_data/sdss/DR12v5/qpm_mocks/mock_galaxy_DR12_CMASS_N_QPM_0002.rdzw
mock_galaxy_DR12_CMASS_N_QPM_0002.rdzw: Read 652204 galaxies from /home/kmsmith/kszx_data/sdss/DR12v5/qpm_mocks/mock_galaxy_DR12_CMASS_N_QPM_0002.rdzw, columns ['ra_deg', 'dec_deg', 'z', 'wfkp', 'wveto']
Redshift cut: zmin=0.43 zmax=0.7: 652204 -> 652204 [100.0% retained]
Writing catalogs/mock_1.h5
mock_galaxy_DR12_CMASS_N_QPM_0002.r